In [1]:
!nvcc --version


/bin/bash: line 1: nvcc: command not found


In [1]:
!nvidia-smi


Sun Dec 28 16:29:01 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# Установка CUDA 12.1 (пример)
!wget https://developer.download.nvidia.com/compute/cuda/12.1.105/local_installers/cuda_12.1.105_535.54.03_linux.run
!chmod +x cuda_12.1.105_535.54.03_linux.run
!./cuda_12.1.105_535.54.03_linux.run --silent --toolkit

# Добавляем в PATH
import os
os.environ['PATH'] += ':/usr/local/cuda-12.1/bin'


--2025-12-28 16:29:16--  https://developer.download.nvidia.com/compute/cuda/12.1.105/local_installers/cuda_12.1.105_535.54.03_linux.run
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 23.32.152.107, 23.32.152.105, 23.32.152.106, ...
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|23.32.152.107|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-12-28 16:29:16 ERROR 404: Not Found.

chmod: cannot access 'cuda_12.1.105_535.54.03_linux.run': No such file or directory
/bin/bash: line 1: ./cuda_12.1.105_535.54.03_linux.run: No such file or directory


In [3]:
!nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [4]:
%%writefile lab4_cuda_merge.cu
// lab4_cuda_merge.cpp
// Параллельная сортировка слиянием на GPU с использованием CUDA
//
// Программа:
// 1) Создаёт массивы размером 10 000 и 100 000 элементов
// 2) Разделяет массив на подмассивы, каждый из которых сортирует отдельный блок
// 3) Параллельно сливает отсортированные подмассивы на GPU
// 4) Замеряет время выполнения и выводит в миллисекундах
//
// Сборка (Linux / Colab):
// nvcc lab4_cuda_merge.cu -O2 -o lab4_cuda_merge
// ./lab4_cuda_merge
//
// Сборка (Windows):
// nvcc lab4_cuda_merge.cu -O2 -o lab4_cuda_merge.exe
// lab4_cuda_merge.exe
//

#include <iostream>
#include <cstdlib>       // Для rand()
#include <cuda_runtime.h>

// Размер блока CUDA
#define BLOCK_SIZE 256

// ===================== Сортировка выбором внутри блока =====================
// Простая функция сортировки подмассива (для демонстрации принципа)
__device__ void selectionSortDevice(int* data, int size) {
    for (int i = 0; i < size-1; i++) {
        int minIdx = i;
        for (int j = i+1; j < size; j++) {
            if (data[j] < data[minIdx]) minIdx = j;
        }
        int tmp = data[i];
        data[i] = data[minIdx];
        data[minIdx] = tmp;
    }
}

// ===================== Сортировка блоков =====================
// Каждый блок GPU сортирует свой подмассив
__global__ void sortBlocks(int* d_data, int subArraySize, int N) {
    int blockStart = blockIdx.x * subArraySize;
    int blockEnd = blockStart + subArraySize;
    if (blockEnd > N) blockEnd = N;

    selectionSortDevice(d_data + blockStart, blockEnd - blockStart);
}

// ===================== Параллельное слияние двух подмассивов =====================
__global__ void mergeArrays(int* d_data, int* d_temp, int width, int N) {
    int idx = blockIdx.x * 2 * width; // индекс первого элемента пары подмассивов

    int left = idx;
    int mid = min(left + width, N);
    int right = min(left + 2*width, N);

    int i = left, j = mid, k = left;
    while (i < mid && j < right) {
        if (d_data[i] <= d_data[j])
            d_temp[k++] = d_data[i++];
        else
            d_temp[k++] = d_data[j++];
    }
    while (i < mid) d_temp[k++] = d_data[i++];
    while (j < right) d_temp[k++] = d_data[j++];
}

// ===================== Главная функция =====================
int main() {
    // Размеры массивов
    int sizes[] = {10000, 100000};

    for (int s = 0; s < 2; s++) {
        int N = sizes[s];

        // ===================== Создаём массив на хосте =====================
        int* h_data = new int[N];
        for (int i = 0; i < N; i++)
            h_data[i] = rand() % 100000;  // случайные числа от 0 до 99999

        // ===================== Выделяем память на GPU =====================
        int* d_data;
        int* d_temp;
        cudaMalloc(&d_data, N * sizeof(int));
        cudaMalloc(&d_temp, N * sizeof(int));

        // Копируем данные на GPU
        cudaMemcpy(d_data, h_data, N * sizeof(int), cudaMemcpyHostToDevice);

        // ===================== Сортировка подмассивов блоками =====================
        int subArraySize = 1024; // размер подмассива для одного блока
        int numBlocks = (N + subArraySize - 1) / subArraySize;

        // Создаём события CUDA для точного замера времени
        cudaEvent_t start, stop;
        cudaEventCreate(&start);
        cudaEventCreate(&stop);
        cudaEventRecord(start);

        // Сортируем каждый блок
        sortBlocks<<<numBlocks, 1>>>(d_data, subArraySize, N);
        cudaDeviceSynchronize();

        // ===================== Параллельное слияние =====================
        // После сортировки подмассивов сливаем их попарно
        for (int width = subArraySize; width < N; width *= 2) {
            int numMergeBlocks = (N + 2*width - 1) / (2*width);
            mergeArrays<<<numMergeBlocks, 1>>>(d_data, d_temp, width, N);
            cudaDeviceSynchronize();

            // Меняем массивы местами
            int* tmp = d_data;
            d_data = d_temp;
            d_temp = tmp;
        }

        // ===================== Замер времени =====================
        cudaEventRecord(stop);
        cudaEventSynchronize(stop);
        float milliseconds = 0;
        cudaEventElapsedTime(&milliseconds, start, stop);

        std::cout << "Array size " << N << " - GPU Merge Sort time: "
                  << milliseconds << " ms" << std::endl;

        // ===================== Освобождаем память =====================
        cudaFree(d_data);
        cudaFree(d_temp);
        delete[] h_data;
    }

    return 0;
}


Writing lab4_cuda_merge.cu


In [21]:
%%writefile lab2_4.cu
// lab2_4.cu
// Параллельная сортировка слиянием на GPU с использованием CUDA
//
// Программа:
// 1) Создаёт массивы размером 10 000 и 100 000 элементов
// 2) Разделяет массив на подмассивы, каждый из которых сортирует отдельный блок
// 3) Параллельно сливает отсортированные подмассивы на GPU
// 4) Замеряет время выполнения и выводит в миллисекундах
//
// Сборка (Windows):
// nvcc lab2_4.cu -O2 -o lab2_4.exe
// lab2_4.exe
//

#include <iostream>         // для std::cout
#include <cstdlib>          // для rand()
#include <cuda_runtime.h>   // CUDA runtime
#include <algorithm>        // для min() в некоторых случаях

// Размер блока CUDA
#define BLOCK_SIZE 256

// ===================== Функция сортировки выбором =====================
// __device__ означает, что функция выполняется на GPU и может вызываться только с ядра (__global__) или другой __device__ функции
__device__ void selectionSortDevice(int* data, int size) {
    // Простейший алгоритм сортировки для небольшого подмассива
    for (int i = 0; i < size-1; i++) {
        int minIdx = i;
        for (int j = i+1; j < size; j++) {
            if (data[j] < data[minIdx])
                minIdx = j;
        }
        // Меняем местами элементы
        int tmp = data[i];
        data[i] = data[minIdx];
        data[minIdx] = tmp;
    }
}

// ===================== Ядро сортировки блоков =====================
// Каждый блок GPU сортирует один подмассив из subArraySize элементов
__global__ void sortBlocks(int* d_data, int subArraySize, int N) {
    int blockStart = blockIdx.x * subArraySize;          // индекс начала подмассива для блока
    int blockEnd = blockStart + subArraySize;           // индекс конца подмассива
    if (blockEnd > N) blockEnd = N;                     // проверка, чтобы не выйти за пределы массива

    // Сортировка подмассива выбором
    selectionSortDevice(d_data + blockStart, blockEnd - blockStart);
}

// ===================== Ядро параллельного слияния =====================
// Каждый блок сливает два соседних подмассива
__global__ void mergeArrays(int* d_data, int* d_temp, int width, int N) {
    int idx = blockIdx.x * 2 * width;                   // индекс первого элемента пары подмассивов

    int left = idx;
    int mid = (left + width < N) ? left + width : N;   // середина подмассива
    int right = (left + 2*width < N) ? left + 2*width : N; // конец второго подмассива

    int i = left, j = mid, k = left;                   // индексы для левого, правого и временного массива
    while (i < mid && j < right) {
        if (d_data[i] <= d_data[j])
            d_temp[k++] = d_data[i++];
        else
            d_temp[k++] = d_data[j++];
    }

    // Копируем оставшиеся элементы левого подмассива
    while (i < mid) d_temp[k++] = d_data[i++];
    // Копируем оставшиеся элементы правого подмассива
    while (j < right) d_temp[k++] = d_data[j++];
}

// ===================== Макрос для проверки ошибок CUDA =====================
#define CUDA_CHECK(err) \
    if (err != cudaSuccess) { \
        std::cerr << "CUDA Error: " << cudaGetErrorString(err) << std::endl; \
        exit(EXIT_FAILURE); \
    }

// ===================== Главная функция =====================
int main() {
    // Определяем размеры массивов, которые будем сортировать
    int sizes[] = {10000, 100000};

    for (int s = 0; s < 2; s++) {
        int N = sizes[s];

        // ===================== Создаём массив на хосте =====================
        int* h_data = new int[N];
        for (int i = 0; i < N; i++)
            h_data[i] = rand() % 100000;  // случайные числа от 0 до 99999

        // ===================== Выделяем память на GPU =====================
        int *d_data, *d_temp;
        CUDA_CHECK(cudaMalloc(&d_data, N * sizeof(int))); // основной массив на GPU
        CUDA_CHECK(cudaMalloc(&d_temp, N * sizeof(int))); // временный массив для слияния

        // Копируем данные с CPU на GPU
        CUDA_CHECK(cudaMemcpy(d_data, h_data, N * sizeof(int), cudaMemcpyHostToDevice));

        // Размер подмассива для одного блока
        int subArraySize = 1024;
        int numBlocks = (N + subArraySize - 1) / subArraySize; // количество блоков

        // ===================== Создаём события CUDA для измерения времени =====================
        cudaEvent_t start, stop;
        CUDA_CHECK(cudaEventCreate(&start));
        CUDA_CHECK(cudaEventCreate(&stop));
        CUDA_CHECK(cudaEventRecord(start));

        // ===================== Сортировка каждого блока =====================
        sortBlocks<<<numBlocks, 1>>>(d_data, subArraySize, N);
        CUDA_CHECK(cudaDeviceSynchronize()); // ожидание завершения сортировки

        // ===================== Параллельное слияние подмассивов =====================
        for (int width = subArraySize; width < N; width *= 2) {
            int numMergeBlocks = (N + 2*width - 1) / (2*width); // количество блоков для слияния
            mergeArrays<<<numMergeBlocks, 1>>>(d_data, d_temp, width, N);
            CUDA_CHECK(cudaDeviceSynchronize());

            // Меняем массивы местами, чтобы следующий шаг слияния работал с новым массивом
            int* tmp = d_data;
            d_data = d_temp;
            d_temp = tmp;
        }

        // ===================== Замер времени =====================
        CUDA_CHECK(cudaEventRecord(stop));
        CUDA_CHECK(cudaEventSynchronize(stop));
        float milliseconds = 0;
        CUDA_CHECK(cudaEventElapsedTime(&milliseconds, start, stop));

        // Выводим время сортировки в миллисекундах
        std::cout << "Array size " << N << " - GPU Merge Sort time: "
                  << milliseconds << " ms" << std::endl;

        // ===================== Освобождение памяти =====================
        CUDA_CHECK(cudaFree(d_data));
        CUDA_CHECK(cudaFree(d_temp));
        delete[] h_data;
    }

    return 0;
}




Overwriting lab2_4.cu


In [22]:
!nvcc lab2_4.cu -O2 -o lab2_4


In [23]:
!./lab2_4


Array size 10000 - GPU Merge Sort time: 7.21062 ms
Array size 100000 - GPU Merge Sort time: 0.01376 ms
